In [1]:
# loading modules
import ee
import geemap

# initialize ee 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()



In [2]:
# CASE 1: Pakistan Floods 2022
# inputs
# Define the Area of Interest (aoi)

name = 'Pakistan_Floods_2022'
aoi = ee.Geometry.Polygon(
    [[[67.98392410136336, 26.049909335428502],
      [67.98392410136336, 25.42892423506662],
      [68.59778518534773, 25.42892423506662],
      [68.59778518534773, 26.049909335428502]]])

startDate = ee.Date('2022-03-01')
endDate = ee.Date('2022-08-20')
predays = 60
postdays = 20

# optional
zvv_value = -3
zvh_val = -3
water_value = 75
elev_value = 800
slope_value = 10
num_samples = 1000

In [3]:
def get_s1_col(date, days, aoi):
    """
    Fetch Sentinel-1 Image Collection based on the given date and filters.

    Parameters:
    date (ee.Date): The starting date for filtering the images.
    days (int): Number of days for filtering the images.
    aoi (ee.Geometry): Area of Interest for filtering the images.

    Returns:
    ee.ImageCollection: Filtered Sentinel-1 Image Collection.
    """
    filters = [
        ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
        ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
        ee.Filter.Or(ee.Filter.equals("instrumentMode", "IW"), ee.Filter.equals("instrumentMode", "SM")),
        ee.Filter.bounds(aoi),
        ee.Filter.eq('resolution_meters', 10),
        ee.Filter.date(date, date.advance(days + 1, 'day'))
    ]
    
    s1_col = ee.ImageCollection('COPERNICUS/S1_GRD').filter(filters)
    return s1_col

# Fetching pre and post-flood images
s1_pre = get_s1_col(startDate, predays, aoi).select(['VV', 'VH'])
s1_post = get_s1_col(endDate, postdays, aoi).select(['VV', 'VH'])

print('Images in S1 Pre: ', s1_pre.size().getInfo())
print('Images in S1 Post: ', s1_post.size().getInfo())

asc = ee.Filter.eq("orbitProperties_pass", "ASCENDING")
des = ee.Filter.eq("orbitProperties_pass", "DESCENDING")

def calc_zscore(s1_pre, s1_post, direction):
    """
    Calculate Z-score for the given direction (ascending/descending).

    Parameters:
    s1_pre (ee.ImageCollection): Pre-flood image collection.
    s1_post (ee.ImageCollection): Post-flood image collection.
    direction (str): Orbit direction (ASCENDING or DESCENDING).

    Returns:
    ee.Image: Z-score image.
    """
    base_mean = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)).mean()
    anom = s1_post.filter(ee.Filter.equals('orbitProperties_pass', direction)).mean().subtract(base_mean)
    base_sd = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)).reduce(ee.Reducer.stdDev()).rename(['VV', 'VH'])
    return anom.divide(base_sd).set({'system:time_start': s1_post.get('system:time_start')})

def zscore_combined():
    """
    Calculate combined Z-score for both ascending and descending orbits.

    Returns:
    ee.Image: Combined Z-score image.
    """
    zscore_des = calc_zscore(s1_pre, s1_post, 'DESCENDING')
    zscore_asc = calc_zscore(s1_pre, s1_post, 'ASCENDING')
    zscore = ee.ImageCollection.fromImages([zscore_des, zscore_asc]).mean().clip(aoi)
    return zscore

def zscore_cond():
    """
    Calculate Z-score based on available orbits (ascending/descending).

    Returns:
    ee.Image: Z-score image based on available orbits.
    """
    cond_asc = s1_pre.filter(asc).size().gt(0).And(s1_post.filter(asc).size().gt(0))
    cond_des = s1_pre.filter(des).size().gt(0).And(s1_post.filter(des).size().gt(0))
    
    if cond_asc.getInfo():
        return calc_zscore(s1_pre, s1_post, 'ASCENDING')
    elif cond_des.getInfo():
        return calc_zscore(s1_pre, s1_post, 'DESCENDING')
    else:
        return ee.Image(0)

def mapFloods(z, aoi, zvv_thd=-2, zvh_thd=-2, pow_thd=75, elev_thd=800, slp_thd=10):
    """
    Generate flood mask based on Z-score and various thresholds.

    Parameters:
    z (ee.Image): Z-score image.
    aoi (ee.Geometry): Area of Interest.
    zvv_thd (float): Threshold for VV band Z-score.
    zvh_thd (float): Threshold for VH band Z-score.
    pow_thd (float): Threshold for open water percentage.
    elev_thd (float): Elevation threshold.
    slp_thd (float): Slope threshold.

    Returns:
    tuple: Flood class and flood layer images.
    """
    # JRC water mask
    jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory").filterDate('2016-01-01', '2022-01-01')
    jrcvalid = jrc.map(lambda x: x.gt(0)).sum()
    jrcwat = jrc.map(lambda x: x.eq(2)).sum().divide(jrcvalid).multiply(100)
    jrcmask = jrcvalid.gt(0)
    ow = jrcwat.gte(ee.Image(pow_thd))

    # Elevation and slope masking using FABDEM
    elevation = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic().setDefaultProjection('EPSG:3857', None, 30)
    slope = ee.Terrain.slope(elevation)

    # Classify floods
    vvflag = z.select('VV').lte(ee.Image(zvv_thd))
    vhflag = z.select('VH').lte(ee.Image(zvh_thd))
    flood_class = ee.Image(0).add(vvflag).add(vhflag.multiply(2)).where(ow.eq(1), 4).rename('flood_class')
    flood_class = flood_class.where(elevation.gt(elev_thd).multiply(ow.neq(1)), 0).where(slope.gt(slp_thd).multiply(ow.neq(1)), 0)

    # Combine flood classes into a single layer
    flood_layer = flood_class.where(flood_class.eq(1), 1).where(flood_class.eq(2), 1).where(flood_class.eq(3), 1).where(flood_class.eq(4), 2)
    flood_layer = flood_layer.where(jrcmask.eq(0), 2).where(flood_class.eq(0), 2).selfMask().rename('label')
    return flood_class.clip(aoi), flood_layer.clip(aoi)

# Determine which Z-score calculation to use
pre_has_asc = s1_pre.filter(asc).size().gt(0)
pre_has_des = s1_pre.filter(des).size().gt(0)
post_has_asc = s1_post.filter(asc).size().gt(0)
post_has_des = s1_post.filter(des).size().gt(0)
cond = pre_has_asc.And(pre_has_des).And(post_has_asc).And(post_has_des)

if cond.getInfo():
    zscore = zscore_combined()
else:
    zscore = zscore_cond()

# Generate flood masks
flood_class, flood_layer = mapFloods(zscore, aoi)


Images in S1 Pre:  30
Images in S1 Post:  11


In [ ]:
def get_s1_col(date, days):
  filters = [
    ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
    ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
    ee.Filter.Or(ee.Filter.equals("instrumentMode", "IW"),ee.Filter.equals("instrumentMode", "SM")),
    ee.Filter.bounds(aoi),
    ee.Filter.eq('resolution_meters', 10),
    ee.Filter.date(date, date.advance(days+1, 'day'))
  ]
  
  s1_col = ee.ImageCollection('COPERNICUS/S1_GRD').filter(filters)

  return s1_col


s1_pre = get_s1_col(startDate, predays).select(['VV', 'VH'])
s1_post = get_s1_col(endDate, postdays).select(['VV', 'VH'])

# images in s1 pre and post
print('Images in S1 Pre: ',s1_pre.size().getInfo())
print('Images in S1 Post: ',s1_post.size().getInfo())

asc = ee.Filter.eq("orbitProperties_pass", "ASCENDING")
des = ee.Filter.eq("orbitProperties_pass", "DESCENDING")

# Function to calculate z-score
def calc_zscore(s1_pre, s1_post, direction):
    base_mean = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)).mean()

    anom = s1_post.filter(ee.Filter.equals('orbitProperties_pass', direction)) \
        .mean().subtract(base_mean).set({'system:time_start': s1_post.get('system:time_start')})

    base_sd = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)) \
        .reduce(ee.Reducer.stdDev()).rename(['VV', 'VH'])

    return anom.divide(base_sd).set({'system:time_start': anom.get('system:time_start')})

def zscore_combined():
  zscore_des = calc_zscore(s1_pre, s1_post,'DESCENDING')
  zscore_asc = calc_zscore(s1_pre, s1_post,'ASCENDING')

  # Calculate mean of z-scores for DESCENDING and ASCENDING orbits
  zscore = ee.ImageCollection.fromImages([zscore_des, zscore_asc]).mean().clip(aoi)

  return zscore

# caculate zscore for only ascending orbits
def zscore_asc():
  base_mean = s1_pre.filter(asc).mean()
  anom = s1_post.filter(asc)\
                .mean().subtract(base_mean)
  base_sd = s1_pre.filter(asc)\
                .reduce(ee.Reducer.stdDev()).rename(['VV', 'VH'])
  zscore = anom.divide(base_sd)
  return zscore

# caculate zscore for only descending orbits
def zscore_des():
  base_mean = s1_pre.filter(des).mean()
  anom = s1_post.filter(des)\
                .mean().subtract(base_mean)
  base_sd = s1_pre.filter(des)\
                .reduce(ee.Reducer.stdDev()).rename(['VV', 'VH'])
  zscore = anom.divide(base_sd)
  return zscore

def zscore_cond():
  cond_a = s1_pre.filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING")).size().gt(0)
  cond_b = s1_pre.filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING")).size().gt(0)
  cond_c = s1_post.filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING")).size().gt(0)
  cond_d = s1_post.filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING")).size().gt(0)
  
  cond_asc = cond_a.And(cond_c)
  cond_des = cond_b.And(cond_d)
  if cond_asc == 1:
    images = zscore_asc()
    zscore = ee.ImageCollection.fromImages([images]).mean().clip(aoi)
  else:
    images = zscore_des()
    zscore = ee.ImageCollection.fromImages([images]).mean().clip(aoi)
  
  return zscore

# Check if both ascending and descending collections are empty
pre_hasAscending = s1_pre.filter([asc]).size().gt(0)
pre_hasDescending = s1_pre.filter([des]).size().gt(0)
post_hasAscending = s1_post.filter([asc]).size().gt(0)
post_hasDescending = s1_post.filter([des]).size().gt(0)

cond = pre_hasAscending.And(pre_hasDescending).And(post_hasAscending).And(post_hasDescending)

if cond == 1:
  zscore = zscore_combined()
else:
  zscore = zscore_cond()


# Flood mask function
def mapFloods(z, zvv_thd=zvv_value, zvh_thd=zvh_val, pow_thd=water_value, elev_thd=elev_value, slp_thd=slope_value):

    # JRC water mask
    jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory").filterDate('2016-01-01', '2022-01-01')
    jrcvalid = jrc.map(lambda x: x.gt(0)).sum()
    jrcwat = jrc.map(lambda x: x.eq(2)).sum().divide(jrcvalid).multiply(100)
    jrcmask = jrcvalid.gt(0)
    ow = jrcwat.gte(ee.Image(pow_thd))

    # Add elevation and slope masking using fabdem
    elevation = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic() \
        .setDefaultProjection('EPSG:3857', None, 30)
    slope = ee.Terrain.slope(elevation)

    # Classify floods
    vvflag = z.select('VV').lte(ee.Image(zvv_thd))
    vhflag = z.select('VH').lte(ee.Image(zvh_thd))

    flood_class = ee.Image(0).add(vvflag) \
        .add(vhflag.multiply(2)) \
        .where(ow.eq(1), 4) \
        .rename('flood_class') \
        .where(elevation.gt(elev_thd).multiply(ow.neq(1)), 0) \
        .where(slope.gt(slp_thd).multiply(ow.neq(1)), 0)
    # add class 1,2,3 from flood_class and create a new layer with single band
    flood_layer = flood_class.where(flood_class.eq(1), 1) \
        .where(flood_class.eq(2), 1) \
        .where(flood_class.eq(3), 1) \
        .where(flood_class.eq(4), 2) \
        .where(jrcmask.eq(0), 2) \
        .where(flood_class.eq(0), 2) \
        .selfMask() \
        .rename('label')
    return flood_class.clip(aoi), flood_layer.clip(aoi)

# Mask z-score threshold
flood_class, flood_layer = mapFloods(zscore)

# sampling



In [ ]:
# visualize flood_class and flood_layer

Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(flood_class, {'min': 0, 'max': 4, 'palette': ['white', 'blue', 'yellow', 'red', 'green']}, 'Flood Class')
Map.addLayer(flood_layer, {'min': 0, 'max': 2, 'palette': ['white', 'red', 'green']}, 'Flood Layer')
Map

In [4]:
split = 0.8  # Train-test split ratio
bands = ['VV', 'VH']  # Bands to use for classification

# Create the 'sample' feature collection
sample = flood_layer.stratifiedSample(
    numPoints=num_samples,
    classBand='label',
    region=aoi,
    scale=10,
    seed=5,
    tileScale=1.5,
    geometries=True
)

# Update label values: change non-flood (label 2) to 0
def updateFeature(feature):
    value = feature.get('label')
    updated_value = ee.Algorithms.If(ee.Algorithms.IsEqual(value, ee.Number(2)), ee.Number(0), value)
    return feature.set('label', updated_value)

label = sample.map(updateFeature)

# Prepare image for classification
image = s1_post.mean().clip(aoi)

# Create training and validation samples
sample_all = image.select(bands).sampleRegions(
    collection=label,
    properties=['label'],
    scale=10
).randomColumn()

training = sample_all.filter(ee.Filter.lt('random', split))
validation = sample_all.filter(ee.Filter.gte('random', split))

# Train the classifier
classifier = ee.Classifier.smileRandomForest(115).train(
    features=training,
    classProperty='label',
    inputProperties=bands
)

# Classify the image
flood_mapped = image.select(bands).classify(classifier)

# Accuracy assessment
train_accuracy = training.classify(classifier).errorMatrix('label', 'classification')
validation_accuracy = validation.classify(classifier).errorMatrix('label', 'classification')

def print_am(error_matrix, dataset_name):
    """
    Print accuracy metrics for a given error matrix.

    Parameters:
    error_matrix (ee.ConfusionMatrix): Error matrix to evaluate.
    dataset_name (str): Name of the dataset (e.g., 'Training', 'Validation').
    """
    overall_accuracy = error_matrix.accuracy().getInfo()
    kappa = error_matrix.kappa().getInfo()
    producers_accuracy = error_matrix.producersAccuracy().getInfo()
    consumers_accuracy = error_matrix.consumersAccuracy().getInfo()
    f1_score = error_matrix.fscore().getInfo()[1]  # F1 score for the flood class (1)

    print(f"{dataset_name} Accuracy Metrics:")
    print(f"Overall Accuracy: {overall_accuracy}")
    print(f"Kappa: {kappa}")
    print(f"Producer's Accuracy: {producers_accuracy}")
    print(f"User's Accuracy: {consumers_accuracy}")
    print(f"F1 Score: {f1_score}")

print_am(train_accuracy, "Training")
print_am(validation_accuracy, "Validation")

# Export results if needed
# geemap.ee_export_image_to_drive(results, description='flood_classification', folder='earth_engine', scale=10, region=aoi)


Training Accuracy Metrics:
Overall Accuracy: 0.9847619047619047
Kappa: 0.9695217822500665
Producer's Accuracy: [[0.985897435897436], [0.9836477987421384]]
User's Accuracy: [[0.9833759590792839, 0.9861286254728878]]
F1 Score: 0.9848866498740555
Validation Accuracy Metrics:
Overall Accuracy: 0.6823529411764706
Kappa: 0.3625354146991835
Producer's Accuracy: [[0.7227272727272728], [0.6390243902439025]]
User's Accuracy: [[0.6824034334763949, 0.6822916666666666]]
F1 Score: 0.6599496221662469


In [ ]:
# test code to compare models

# Parameters
num_samples = 1000  # Number of sample points
split = 0.8  # Train-test split ratio
bands = ['VV', 'VH']  # Bands to use for classification

# Create the 'sample' feature collection
sample = flood_layer.stratifiedSample(
    numPoints=num_samples,
    classBand='label',
    region=aoi,
    scale=10,
    seed=5,
    tileScale=1.5,
    geometries=True
)

# Update label values: change non-flood (label 2) to 0
def updateFeature(feature):
    value = feature.get('label')
    updated_value = ee.Algorithms.If(ee.Algorithms.IsEqual(value, ee.Number(2)), ee.Number(0), value)
    return feature.set('label', updated_value)

label = sample.map(updateFeature)

# Verify label values
unique_labels = label.aggregate_array('label').distinct().getInfo()
print(f"Unique labels: {unique_labels}")

# Prepare image for classification
image = s1_post.mean().clip(aoi)

# Create training and validation samples
sample_all = image.select(bands).sampleRegions(
    collection=label,
    properties=['label'],
    scale=10
).randomColumn()

training = sample_all.filter(ee.Filter.lt('random', split))
validation = sample_all.filter(ee.Filter.gte('random', split))

# Define classifiers to compare
classifiers = {
    'RandomForest': ee.Classifier.smileRandomForest(115),
    'GradientTreeBoost': ee.Classifier.smileGradientTreeBoost(100),
    'DecisionTree': ee.Classifier.smileCart(),
    #'NaiveBayes': ee.Classifier.smileNaiveBayes(),
    'SVM': ee.Classifier.libsvm()
}

def evaluate_classifier(classifier, training, validation):
    start_time = time.time()
    
    trained_classifier = classifier.train(
        features=training,
        classProperty='label',
        inputProperties=bands
    )
    
    training_classified = training.classify(trained_classifier)
    validation_classified = validation.classify(trained_classifier)
    
    training_accuracy = training_classified.errorMatrix('label', 'classification')
    validation_accuracy = validation_classified.errorMatrix('label', 'classification')
    
    end_time = time.time()
    prediction_time = end_time - start_time
    
    return {
        'train_f1': training_accuracy.fscore().getInfo()[1] if len(training_accuracy.fscore().getInfo()) > 1 else None,
        'validation_f1': validation_accuracy.fscore().getInfo()[1] if len(validation_accuracy.fscore().getInfo()) > 1 else None,
        'train_producer_accuracy': training_accuracy.producersAccuracy().getInfo()[1] if len(training_accuracy.producersAccuracy().getInfo()) > 1 else None,
        'validation_producer_accuracy': validation_accuracy.producersAccuracy().getInfo()[1] if len(validation_accuracy.producersAccuracy().getInfo()) > 1 else None,
        'train_consumer_accuracy': training_accuracy.consumersAccuracy().getInfo()[1] if len(training_accuracy.consumersAccuracy().getInfo()) > 1 else None,
        'validation_consumer_accuracy': validation_accuracy.consumersAccuracy().getInfo()[1] if len(validation_accuracy.consumersAccuracy().getInfo()) > 1 else None,
        'prediction_time': prediction_time
    }

# Evaluate all classifiers
results = {}
for name, classifier in classifiers.items():
    print(f"Evaluating {name} classifier...")
    results[name] = evaluate_classifier(classifier, training, validation)

# Print results
for name, metrics in results.items():
    print(f"\nClassifier: {name}")
    print(f"Training F1 Score: {metrics['train_f1']}")
    print(f"Validation F1 Score: {metrics['validation_f1']}")
    print(f"Training Producer Accuracy: {metrics['train_producer_accuracy']}")
    print(f"Validation Producer Accuracy: {metrics['validation_producer_accuracy']}")
    print(f"Training Consumer Accuracy: {metrics['train_consumer_accuracy']}")
    print(f"Validation Consumer Accuracy: {metrics['validation_consumer_accuracy']}")
    print(f"Prediction Time: {metrics['prediction_time']} seconds")

# Optionally, export the best classifier's results
# best_classifier_name = max(results, key=lambda k: results[k]['validation_f1'])
# best_classifier = classifiers[best_classifier_name].train(
#     features=training,
#     classProperty='label',
#     inputProperties=bands
# )
# classified_image = image.select(bands).classify(best_classifier)
# geemap.ee_export_image_to_drive(classified_image, description=f'best_classifier_{best_classifier_name}', folder='earth_engine', scale=10, region=aoi)


# Susceptibility

In [5]:
# Variables Preparation for Susceptibility
# Variables used: DEM, Slope, Aspect, NDVI, NDWI, NDBI, NDSI

# Apply scaling factors to Landsat images
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Year extraction from endDate
year = ee.Date(endDate).get('year')

# Filter Landsat 8 for 2013 to end of 2021
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        .filterBounds(aoi)\
        .filterDate('2013-01-01', '2021-11-01')

# Filter Landsat 9 for 2022 onwards
l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
        .filterBounds(aoi)

# Combine both collections, prioritizing Landsat 9 for 2022
landsat_combined = l8.merge(l9)

# Filter the combined collection for the specified year
landsat_filtered = landsat_combined.filter(ee.Filter.calendarRange(year, year, 'year'))\
                                   .filter(ee.Filter.lt('CLOUD_COVER', 10))\
                                   .map(applyScaleFactors)\
                                   .median()\
                                   .clip(aoi)
proj = landsat_filtered.projection()
# Datasets
dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM")\
        .filterBounds(aoi)\
        .mosaic()\
        .clip(aoi).reproject('EPSG:3395', None, 30).rename('elevation')

dem_proj = dem.setDefaultProjection(proj)

slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)


slope_proj = slope.setDefaultProjection(proj)
aspect_proj = aspect.setDefaultProjection(proj)


ndvi = landsat_filtered.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndwi = landsat_filtered.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
ndbi = landsat_filtered.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')
ndsi = landsat_filtered.normalizedDifference(['SR_B2', 'SR_B5']).rename('NDSI')

landsat_filtered = landsat_filtered.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

rainfall = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
            .filterDate('2018-01-01', '2023-01-01')\
            .filterBounds(aoi)\
            .map(lambda image: image.gt(10).selfMask())\
            .map(lambda image: image.clip(aoi))\
            .sum().rename('rainfall')\
            #.reproject(proj.crs(), None, 30)

# Combine all layers into one image
image_sus = landsat_filtered.addBands(ndvi).addBands(ndwi).addBands(ndbi).addBands(ndsi)\
                            .addBands(dem_proj).addBands(slope_proj).addBands(aspect_proj)\
                            .addBands(rainfall)\
                            .clip(aoi)

# Get band names
bands_sus = image_sus.bandNames().getInfo()

# Create the flood labels based on ML classified flood layer

sample_new = flood_mapped.rename('label').stratifiedSample(
    numPoints=num_samples,
    classBand='label',
    region=aoi,
    scale=30,
    seed=5,
    tileScale=1.5,
    geometries=True
)

# Update label values: change non-flood (label 2) to 0
def updateFeature(feature):
    value = feature.get('label')
    updated_value = ee.Algorithms.If(ee.Algorithms.IsEqual(value, ee.Number(2)), ee.Number(0), value)
    return feature.set('label', updated_value)

label_new = sample_new.map(updateFeature)


# Training sample collection
sample_all_sus = image_sus.select(bands_sus).clip(aoi).sampleRegions(
    collection=label_new,
    properties=['label'],
    scale=30,
    tileScale=1.5
).randomColumn()

# size of sample_all_sus
print('Size of sample_all_sus: ', sample_all_sus.size().getInfo())


training_sus = sample_all_sus.filter(ee.Filter.lt('random', split))
validation_sus = sample_all_sus.filter(ee.Filter.gte('random', split))

# Train the classifier
classifier_sus = ee.Classifier.smileRandomForest(115).train(
    features=training_sus,
    classProperty='label',
    inputProperties=bands_sus
)

# Set the classifier to output probabilities
classifier_prob = classifier_sus.setOutputMode('PROBABILITY')

# Classify the image to get flood susceptibility probabilities
flood_prob = image_sus.classify(classifier_prob)

# Accuracy assessment
def calculate_metrics(validation, classifier):
    validation_classified = validation.classify(classifier)
    validation_accuracy = validation_classified.errorMatrix('label', 'classification')
    
    f1_score = validation_accuracy.fscore().getInfo()[1] if len(validation_accuracy.fscore().getInfo()) > 1 else None
    producer_accuracy = validation_accuracy.producersAccuracy().getInfo()[1] if len(validation_accuracy.producersAccuracy().getInfo()) > 1 else None
    consumer_accuracy = validation_accuracy.consumersAccuracy().getInfo()[1] if len(validation_accuracy.consumersAccuracy().getInfo()) > 1 else None
    
    return f1_score, producer_accuracy, consumer_accuracy

f1_score, producer_accuracy, consumer_accuracy = calculate_metrics(validation_sus, classifier_sus)

print("Validation F1 Score:", f1_score)
print("Validation Producer Accuracy:", producer_accuracy)
print("Validation Consumer Accuracy:", consumer_accuracy)



ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [16]:
# get feature importance from model
feature_importance = classifier_sus.explain().get('importance')
feature_importance.getInfo()


{'NDBI': 274.529298710548,
 'NDSI': 287.53693181946596,
 'NDVI': 274.61236565608283,
 'NDWI': 275.2941965118867,
 'SR_B1': 274.5565817940799,
 'SR_B2': 282.46621514574116,
 'SR_B3': 289.9543859135285,
 'SR_B4': 271.74068040489,
 'SR_B5': 270.7042220065041,
 'SR_B6': 290.6287738247068,
 'SR_B7': 253.48013290668456,
 'aspect': 285.7636875825801,
 'elevation': 279.5141124562158,
 'rainfall': 197.11784528261626,
 'slope': 300.8232063920606}

In [17]:
# visualize flood layer and flood susceptibility probabilities
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(flood_mapped, {'min': 0, 'max': 2, 'palette': ['white', 'blue', 'green']}, 'Flood Layer')
Map.addLayer(flood_prob, {'min': 0, 'max': 1, 'palette': ['#006837', ' #3ba858', ' #9dd569', ' #e4f49a', ' #fee99a', ' #fca55d', ' #e34a33', ' #a50026']}, 'Flood Susceptibility Probabilities')
Map

Map(center=[25.73946796510279, 68.29085464335516], controls=(WidgetControl(options=['position', 'transparent_b…

In [42]:

# Apply scaling factors to Landsat images
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Year extraction from endDate
year = ee.Date(endDate).get('year')

# Filter Landsat 8 for 2013 to end of 2021
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        .filterBounds(aoi)\
        .filterDate('2013-01-01', '2021-12-31')

# Filter Landsat 9 for 2022 onwards
l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
        .filterBounds(aoi)\
        .filterDate('2022-01-01', endDate)

# Apply scaling factors
l8 = l8.map(applyScaleFactors)
l9 = l9.map(applyScaleFactors)

# Combine both collections, prioritizing Landsat 9 for 2022
landsat_combined = l8.merge(l9)

# Filter the combined collection for the specified year
landsat_filtered = landsat_combined.filter(ee.Filter.calendarRange(year, year, 'year'))\
                                   .filter(ee.Filter.lt('CLOUD_COVER', 40))\
                                   .median()\
                                   .clip(aoi)\
                                   .reproject('EPSG:3395', None, 30)

# Datasets
dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM")\
        .filterBounds(aoi)\
        .mosaic()\
        .clip(aoi)\
        .reproject('EPSG:3395', None, 30)

slope = ee.Terrain.slope(dem).reproject('EPSG:3395', None, 30)
aspect = ee.Terrain.aspect(dem).reproject('EPSG:3395', None, 30)

ndvi = landsat_filtered.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI').reproject('EPSG:3395', None, 30)
ndwi = landsat_filtered.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI').reproject('EPSG:3395', None, 30)
ndbi = landsat_filtered.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI').reproject('EPSG:3395', None, 30)
ndsi = landsat_filtered.normalizedDifference(['SR_B2', 'SR_B5']).rename('NDSI').reproject('EPSG:3395', None, 30)

rainfall = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
            .filterDate('2018-01-01', '2023-01-01')\
            .filterBounds(aoi)\
            .map(lambda image: image.gt(10).selfMask())\
            .map(lambda image: image.clip(aoi))\
            .sum().rename('rainfall')\
            .reproject('EPSG:3395', None, 30)

landsat_filtered = landsat_filtered.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

# Combine all layers into one image
image_sus = landsat_filtered.addBands(dem).addBands(slope).addBands(aspect)\
                            .addBands(ndvi).addBands(ndwi).addBands(ndbi).addBands(ndsi)\
                            .addBands(rainfall)\
                            .clip(aoi)

# Get band names
bands_sus = image_sus.bandNames().getInfo()

# Reproject labels to EPSG:3395
label = ee.FeatureCollection(label).map(lambda feature: feature.setGeometry(feature.geometry().transform('EPSG:3395')))

# Check if the label collection has valid data
label_size = label.size().getInfo()
print('Size of label collection:', label_size)

# Check projections
image_proj = image_sus.projection().getInfo()
feature_proj = label.first().geometry().projection().getInfo()
print("Image Projection Information:")
print(image_proj)
print("\nFeature Collection Projection Information:")
print(feature_proj)

# Training sample collection
sample_all_sus = image_sus.select(bands_sus).sampleRegions(
    collection=label,
    properties=['label'],
    scale=30,
    tileScale=2
)

# Print size of sample_all_sus
sample_size = sample_all_sus.size().getInfo()
print('Size of sample_all_sus:', sample_size)

if sample_size == 0:
    # Debug information if no samples are found
    print('No samples found. Check the intersection of the label and image_sus.')
    
    # Get the bounding box of the aoi and label geometries
    aoi_bbox = aoi.bounds().getInfo()
    label_bbox = label.geometry().bounds().getInfo()
    print("AOI Bounding Box:", aoi_bbox)
    print("Label Bounding Box:", label_bbox)

# Continue with the random column and rest of the code only if samples are found
if sample_size > 0:
    sample_all_sus = sample_all_sus.randomColumn()

    split = 0.9
    training_sus = sample_all_sus.filter(ee.Filter.lt('random', split))
    validation_sus = sample_all_sus.filter(ee.Filter.gte('random', split))

    # Print sizes of training and validation sets
    training_size = training_sus.size().getInfo()
    validation_size = validation_sus.size().getInfo()
    print(f"Training set size: {training_size}")
    print(f"Validation set size: {validation_size}")

    # Ensure there are valid training and validation samples
    if training_size == 0 or validation_size == 0:
        raise ValueError("No valid training or validation samples found. Check the sampling and filtering steps.")

    # Train the classifier
    classifier_sus = ee.Classifier.smileRandomForest(115).train(
        features=training_sus,
        classProperty='label',
        inputProperties=bands_sus
    )

    # Set the classifier to output probabilities
    classifier_prob = classifier_sus.setOutputMode('PROBABILITY')

    # Classify the image to get flood susceptibility probabilities
    results_prob = image_sus.classify(classifier_prob)

    # Display the results on the map
    Map = geemap.Map()
    Map.centerObject(aoi, 10)
    Map.addLayer(results_prob, {'min': 0, 'max': 1, 'palette': ['green', 'blue', 'red']}, 'Flood Susceptibility')
    Map

    # Accuracy assessment
    def calculate_metrics(validation, classifier):
        validation_classified = validation.classify(classifier)
        validation_accuracy = validation_classified.errorMatrix('label', 'classification')

        f1_score = validation_accuracy.fscore().getInfo()[1] if len(validation_accuracy.fscore().getInfo()) > 1 else None
        producer_accuracy = validation_accuracy.producersAccuracy().getInfo()[1] if len(validation_accuracy.producersAccuracy().getInfo()) > 1 else None
        consumer_accuracy = validation_accuracy.consumersAccuracy().getInfo()[1] if len(validation_accuracy.consumersAccuracy().getInfo()) > 1 else None

        return f1_score, producer_accuracy, consumer_accuracy

    f1_score, producer_accuracy, consumer_accuracy = calculate_metrics(validation_sus, classifier_sus)

    print("Validation F1 Score:", f1_score)
    print("Validation Producer Accuracy:", producer_accuracy)
    print("Validation Consumer Accuracy:", consumer_accuracy)
else:
    print("No samples found. Please check the data and ensure the label and image_sus intersect spatially and temporally.")


Size of label collection: 2000
Image Projection Information:
{'type': 'Projection', 'crs': 'EPSG:3395', 'transform': [30, 0, 0, 0, -30, 0]}

Feature Collection Projection Information:
{'type': 'Projection', 'crs': 'EPSG:3395', 'transform': [1, 0, 0, 0, 1, 0]}
Size of sample_all_sus: 1998
Training set size: 1808
Validation set size: 190
Validation F1 Score: 0.736842105263158
Validation Producer Accuracy: [0.719626168224299]
Validation Consumer Accuracy: None


In [43]:
    # Display the results on the map
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(results_prob, {'min': 0, 'max': 1, 'palette': ['green', 'blue', 'red']}, 'Flood Susceptibility')
Map

Map(center=[25.73946796510279, 68.29085464335516], controls=(WidgetControl(options=['position', 'transparent_b…

EEException: User memory limit exceeded.

In [45]:
results_prob.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [2279, 2547],
   'origin': [252264, -99552],
   'crs': 'EPSG:3395',
   'crs_transform': [30, 0, 0, 0, -30, 0]}],
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[67.98392410136336, 25.42892423506662],
     [68.59778518534773, 25.42892423506662],
     [68.59778518534773, 26.049909335428502],
     [67.98392410136336, 26.049909335428502],
     [67.98392410136336, 25.42892423506662]]]}}}

In [47]:
# scale image between 1 to 100 (integers) and export
results_prob_scaled = results_prob.multiply(100).toInt()
geemap.ee_export_image(results_prob_scaled, filename='flood_susceptibility.tif', scale=30, region=aoi, file_per_band=False)

Generating URL ...
Please wait ...
An error occurred while downloading.


In [ ]:

training_sus = sample_all_sus.filter(ee.Filter.lt('random', split))
validation_sus = sample_all_sus.filter(ee.Filter.gte('random', split))

# Train the classifier
classifier_sus = ee.Classifier.smileRandomForest(115).train(
    features=training_sus,
    classProperty='label',
    inputProperties=bands_sus
)

# Set the classifier to output probabilities
classifier_prob = classifier_sus.setOutputMode('PROBABILITY')

# Classify the image to get flood susceptibility probabilities
results_prob = image_sus.classify(classifier_prob)

# Accuracy assessment
def calculate_metrics(validation, classifier):
    validation_classified = validation.classify(classifier)
    validation_accuracy = validation_classified.errorMatrix('label', 'classification')
    
    f1_score = validation_accuracy.fscore().getInfo()[1] if len(validation_accuracy.fscore().getInfo()) > 1 else None
    producer_accuracy = validation_accuracy.producersAccuracy().getInfo()[1] if len(validation_accuracy.producersAccuracy().getInfo()) > 1 else None
    consumer_accuracy = validation_accuracy.consumersAccuracy().getInfo()[1] if len(validation_accuracy.consumersAccuracy().getInfo()) > 1 else None
    
    return f1_score, producer_accuracy, consumer_accuracy

f1_score, producer_accuracy, consumer_accuracy = calculate_metrics(validation_sus, classifier_sus)

print("Validation F1 Score:", f1_score)
print("Validation Producer Accuracy:", producer_accuracy)
print("Validation Consumer Accuracy:", consumer_accuracy)


In [29]:
# function to print crs of all bands of sus image

def print_crs(image):
    """
    Print the CRS of all bands in an image.

    Parameters:
    image (ee.Image): The image to evaluate.
    """
    for band in image.bandNames().getInfo():
        crs = image.select(band).projection().getInfo()['crs']
        print(f"{band} CRS: {crs}")
        
print_crs(image_sus)

SR_B1 CRS: EPSG:4326
SR_B2 CRS: EPSG:4326
SR_B3 CRS: EPSG:4326
SR_B4 CRS: EPSG:4326
SR_B5 CRS: EPSG:4326
SR_B6 CRS: EPSG:4326
SR_B7 CRS: EPSG:4326
b1 CRS: EPSG:4326
slope CRS: EPSG:4326
aspect CRS: EPSG:4326
NDVI CRS: EPSG:4326
NDWI CRS: EPSG:4326
NDBI CRS: EPSG:4326
NDSI CRS: EPSG:4326


In [ ]:
# Display the results on the map
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(results_prob, {'min': 0, 'max': 1, 'palette': ['green', 'blue', 'red']}, 'Flood Susceptibility')
Map

In [ ]:
# Variables Preperation for Susceptibility


# varianles used DEM, Slope, Aspect, NDVI, NDWI, NDBI, NDSI

# Applies scaling factors.
year = endDate.get('year')
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    return image.addBands(opticalBands, None, True)\
              .addBands(thermalBands, None, True)

def remapper(image):
        remapped = image.remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9])
        return remapped

# datasets
dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM")\
        .filterBounds(aoi)\
        .mosaic()\
        .clip(aoi)

slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)


l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
        .filterBounds(aoi)\
        .filter(ee.Filter.calendarRange(year, year, 'year'))\
        .map(applyScaleFactors)\
        .filter(ee.Filter.lt('CLOUD_COVER', 10))\
        .median()\
        .clip(aoi)


ndvi = l9.normalizedDifference(['SR_B5', 'SR_B4'])
ndwi = l9.normalizedDifference(['SR_B3', 'SR_B5'])
ndbi = l9.normalizedDifference(['SR_B6', 'SR_B5'])
ndsi = l9.normalizedDifference(['SR_B2', 'SR_B5'])

l9 = l9.select(['SR_B1','SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

image_sus = l9.addBands(dem).addBands(slope).addBands(aspect).addBands(ndvi).addBands(ndwi).addBands(ndbi).addBands(ndsi).clip(aoi)

bands_sus = image_sus.bandNames().getInfo()

# training
sample_all_sus = image_sus.select(bands_sus).sampleRegions(**{
    'collection': label,
    'properties': ['label'],
    'scale': 30
})

sample_all_sus = sample_all_sus.randomColumn()

split = 0.9
training_sus = sample_all_sus.filter(ee.Filter.lt('random', split))
validation_sus = sample_all_sus.filter(ee.Filter.gte('random', split))


classifier_sus = ee.Classifier.smileRandomForest(115).train(**{
    'features': training_sus,
    'classProperty': 'label',
    'inputProperties': bands_sus
})

classifier_prob = classifier_sus.setOutputMode('PROBABILITY')
results_prob = image_sus.select(bands).classify(classifier_prob)
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(results_prob, {'min': 0, 'max': 1, 'palette': ['green', 'blue', 'red']}, 'Flood Susceptibility');


In [ ]:
Map.addLayer(results, {'min': 1, 'max': 3, 'palette': ['white', 'red', '233CF0']}, 'Flood Classified');
Map.addLayer(results_prob, {'min': 0, 'max': 1, 'palette': ['white', 'blue', '233CF0']}, 'Flood Susceptibility');

# map
Map = geemap.Map()
Map.centerObject(aoi)
Map.addLayer(aoi, {}, 'Area of Interest')
Map.addLayer(zscore.select('VV'), {'min': -7, 'max': 7, 'palette': ['red', 'white', 'blue']}, 'ZScore', False)
Map.addLayer(flood_class, {'min': 0, 'max': 4, 'palette': ['#E3E3E3','#FFB100', '#FFB100', '#3E9DFF','#031DC9']}, 'flood classes', False)
Map.addLayer(flood_layer, {'min': 1, 'max': 2, 'palette': ['blue', 'white']}, 'flood layer', False)
Map.addLayer(label, {}, 'Label')
Map

In [ ]:
classifier_explain = classifier.explain()
variable_importance = ee.Feature(None, ee.Dictionary(classifier_explain).get('importance'))
#print('Classifier Explain: ', classifier_explain)
print('Variable Importance: ', variable_importance.getInfo())
train_accuracy = classifier.confusionMatrix()
print('Train Accuracy', train_accuracy.getInfo())
print('Overall Accuracy: ', train_accuracy.accuracy().getInfo())
print('Kappa: ', train_accuracy.kappa().getInfo())
print('Producer Accuracy: ', train_accuracy.producersAccuracy().getInfo())
print('Consumer Accuracy: ', train_accuracy.consumersAccuracy().getInfo())

validated = validation.classify(classifier)
print('Validation', validated.first().getInfo())
test_accuracy = validated.errorMatrix('label', 'classification')
print('Validation Accuracy', test_accuracy.getInfo())
print('Validation Overall Accuracy: ', test_accuracy.accuracy().getInfo())
print('Validation Kappa: ', test_accuracy.kappa().getInfo())
print('Validation Producer Accuracy: ', test_accuracy.producersAccuracy().getInfo())
print('Validation Consumer Accuracy: ', test_accuracy.consumersAccuracy().getInfo())

f1_train = train_accuracy.fscore().getInfo()[1]
print('F1 Score Train: ', f1_train)

f1_test = test_accuracy.fscore().getInfo()[1]
print('F1 Score Test: ', f1_test)



In [ ]:
# exporting
geemap.ee_to_shp(label, filename=f'../data/{name}_label.shp')
#geemap.ee_export_image_to_drive(flood_layer, description='flood_layer', folder='s1_post', region=aoi, scale=10)